<a href="https://colab.research.google.com/github/cedro3/Style_edit/blob/main/Style_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ1（潜在変数の推定）

In [ ]:
# 1.tensorflow & Pytorch バージョン変更
%tensorflow_version 1.x
! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html

# 2.githubからコードを取得 & ninja インストール
import os
os.chdir('/content')
CODE_DIR = 'Style_edit' 
!git clone https://github.com/cedro3/Style_edit.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

# 3.pSpインストール
sys.path.append(".")
sys.path.append("..")
from utils.common import tensor2im
from models.psp import pSp  
%load_ext autoreload
%autoreload 2

# 4.学習済みパラメータのダウンロード
import os
import gdown
os.makedirs('pretrained_models', exist_ok=True)
gdown.download('https://drive.google.com/u/0/uc?id=1cUv_reLE6k3604or78EranS7XzuVMWeO', 'pretrained_models/e4e_ffhq_encode.pt', quiet=False)

# 5.ランドマークデータのダウンロード
! wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
! bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

# 6.モデルに学習済みパラメータをロード
model_path = 'pretrained_models/e4e_ffhq_encode.pt'  
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

# セットアップ2（潜在変数の編集）

In [ ]:
# 1.CLIPインストール
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

# 2.学習済みパラメータのダウンロード
gdown.download('https://drive.google.com/u/1/uc?id=13CCGLcCw6_GMHe8cUBiaLlORzEK4gwso', 'data_sc.zip', quiet=False)
! unzip data_sc.zip

import tensorflow as tf
import numpy as np 
import torch
import clip
from PIL import Image
import pickle
import copy
import matplotlib.pyplot as plt
from MapTS import GetFs,GetBoundary,GetDt
from manipulate import Manipulator

# 3.CLIPのモデル化
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 4.モデルに学習済みパラメータをロード
M=Manipulator(dataset_name='ffhq')
fs3=np.load('./npy/ffhq/fs3.npy')
np.set_printoptions(suppress=True)

# 顔画像の切り出し

In [ ]:
# --- 画像表示関数 ---
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = os.listdir(folder)
    files.sort()
    for i, file in enumerate(files):
        if file=='.ipynb_checkpoints':
           continue
        img = Image.open(folder+'/'+file)    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(folder+'/'+file, fontsize=15)               
    plt.show()
    plt.close()  

In [ ]:
# --- サンプル画像表示 ---
display_pic('images')

In [ ]:
# --- 顔画像の切り出し ---
import os
import shutil
from tqdm import tqdm

if os.path.isdir('align'):
     shutil.rmtree('align')
os.makedirs('align', exist_ok=True)

def run_alignment(image_path):
  import dlib
  from utils.alignment import align_face
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor) 
  return aligned_image 

files = sorted(os.listdir('./images'))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = run_alignment('./images/'+file)
  input_image.resize((256,256))
  input_image.save('./align/'+file)

display_pic('align')

# 潜在変数の推定

In [ ]:
# --- 潜在変数の推定 ---
if os.path.isdir('vec_pic'):
     shutil.rmtree('vec_pic')
os.makedirs('vec_pic', exist_ok=True)

if os.path.isdir('vec'):
     shutil.rmtree('vec')
os.makedirs('vec', exist_ok=True)

img_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

files = sorted(os.listdir('./align'))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = Image.open('./align/'+file)
  transformed_image = img_transforms(input_image)
  with torch.no_grad():
     images, latents = net(transformed_image.unsqueeze(0).to('cuda').float(), randomize_noise=False, return_latents=True)
     result_image, latent = images[0], latents[0]
     tensor2im(result_image).save('./vec_pic/'+file) 
     torch.save(latents, './vec/'+file[:-4]+'.pt') 

display_pic('vec_pic')

In [ ]:
# --- 実写と潜在変数の比較 ---
display_pic('align')
display_pic('vec_pic')

# 潜在変数の編集

In [ ]:
# --- 潜在変数の選択 ---
pt_folder = 'vec/'
pt_name = '003.pt' #@param {type:"string"}
latents=torch.load(pt_folder+pt_name)
w_plus=latents.cpu().detach().numpy()
M.dlatents=M.W2S(w_plus)

M.num_images=1
M.alpha=[0]
M.manipulate_layers=[0]
codes,out=M.EditOneC(0,M.dlatents) 
original=Image.fromarray(out[0,0]).resize((512,512))
M.manipulate_layers=None
original

In [ ]:
# --- 編集テキスト入力 ---
neutral='face' #@param {type:"string"}
target='smiling face' #@param {type:"string"}
classnames=[target,neutral]
dt=GetDt(classnames,model)

In [ ]:
# --- alpha & beta の設定 ---
beta = 0.1 #@param {type:"slider", min:0.08, max:0.3, step:0.01}
alpha = 2 #@param {type:"slider", min:-10, max:10, step:0.1}
M.alpha=[alpha]
boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
codes=M.MSCode(M.dlatents,boundary_tmp2)
out=M.GenerateImg(codes)
generated=Image.fromarray(out[0,0])#.resize((512,512))
generated.save('generated.jpg')

plt.figure(figsize=(14,7), dpi= 100)
plt.subplot(1,2,1)
plt.imshow(original)
plt.title('original')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(generated)
plt.title('manipulated')
plt.axis('off')